## This just needs the images from the 'df_train_preprocessed_clip_labels.csv' in 'path' column to be present in the method stated 

### Install Dependencies

In [ ]:
#pip install ftfy regex tqdm
#pip install git+https://github.com/openai/CLIP.git
#pip install scikit-learn pandas matplotlib tensorflow


### Imports

In [ ]:
import os
import clip
import torch
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm


### Load and Split Dataset

In [ ]:
# Load your CSV file
df = pd.read_csv("df_train_preprocessed_clip_labels.csv") 

# Drop missing paths or captions
df = df.dropna(subset=['path', 'caption_concise'])

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


### Load CLIP and Define Embedding Function

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_clip_embeddings(dataframe):
    features = []
    labels = []

    for idx, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        try:
            image = preprocess(Image.open(row['path']).convert("RGB")).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)
            features.append(image_features.cpu().numpy())
            labels.append(row['caption_concise'])
        except Exception as e:
            print(f"Error on {row['path']}: {e}")
    
    return np.vstack(features), labels


### Generate Embeddings

In [ ]:

X_train, y_train_text = get_clip_embeddings(train_df)
X_test, y_test_text = get_clip_embeddings(test_df)


### Encode Labels

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train_text)
y_test = le.transform(y_test_text)

class_names = le.classes_


### Create TensorFlow Datasets

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(500).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)


### Build and Compile Classifier Model

In [ ]:
model_tf = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(512,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model_tf.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


### Train the Model

In [ ]:
history = model_tf.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


### Plot Accuracy and Loss

In [ ]:
# Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


### Repeat & Average Multiple Runs

In [ ]:
accuracies = []

for seed in range(5):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)
    
    X_train, y_train_text = get_clip_embeddings(train_df)
    X_test, y_test_text = get_clip_embeddings(test_df)
    
    y_train = le.fit_transform(y_train_text)
    y_test = le.transform(y_test_text)
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(500).batch(32)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)
    
    model_tf = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(512,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])
    
    model_tf.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
    
    history = model_tf.fit(train_ds, validation_data=test_ds, epochs=10, verbose=0)
    
    acc = history.history['val_accuracy'][-1]
    accuracies.append(acc)
    print(f"Run {seed+1} Accuracy: {acc:.4f}")

print(f"\nAverage Accuracy: {np.mean(accuracies):.4f}")
